<a href="https://colab.research.google.com/github/Vurimindi2021/CNN-to-Detect-Atelectasis-in-Chest-X-Ray/blob/main/src/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Python Libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import shutil
import matplotlib.pyplot as plt
from os import path
from matplotlib.image import imread
from google.colab import drive
from keras.preprocessing.image import ImageDataGenerator, load_img

In [ ]:
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
csvlst= ['/content/drive/My Drive/Colab Notebooks/images_01.csv',
         '/content/drive/My Drive/Colab Notebooks/images_02.csv',
         '/content/drive/My Drive/Colab Notebooks/images_03.csv',
         '/content/drive/My Drive/Colab Notebooks/images_04.csv',
         '/content/drive/My Drive/Colab Notebooks/images_05.csv',
         '/content/drive/My Drive/Colab Notebooks/images_06.csv',
         '/content/drive/My Drive/Colab Notebooks/images_07.csv',
         '/content/drive/My Drive/Colab Notebooks/images_08.csv',
         '/content/drive/My Drive/Colab Notebooks/images_09.csv',
         '/content/drive/My Drive/Colab Notebooks/images_10.csv',
         '/content/drive/My Drive/Colab Notebooks/images_11.csv',
         '/content/drive/My Drive/Colab Notebooks/images_12.csv'
        ]

In [ ]:
imported_images = pd.concat(map(pd.read_csv, csvlst))
imported_images.drop(columns=["Unnamed: 0"])
image_metadata = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data_Entry_2017_v2020.csv')
bbox_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BBox_List_2017.csv')
print("Images Metadata: ", len(image_metadata), "Imported Inages: ", len(imported_images), "BBox Images: ", len(bbox_data))

In [ ]:
splitfindings = image_metadata['Finding Labels'].str.split("|", expand = True)
splitfindings.info()
image_metadata['Finding'] = splitfindings[0]
image_metadata['Finding1']=image_metadata['Finding'].str.replace(" ","")
image_metadata['Finding1'].value_counts()

In [ ]:
merge_image_metadata= pd.merge(
    right=imported_images,
    left=image_metadata,
    how="inner",
    on=None,
    right_on=['Image Index'],
    left_on=['Image Index'],
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=None,
    copy=True,
    indicator=False,
    validate=None)
print("Image Metadata:",len(image_metadata))
print("Imported Images:",len(imported_images))
print("Merged:", len(merge_image_metadata))

In [ ]:
parent_dir= "/content/drive/MyDrive/images/"
dirlst=list(image_metadata['Finding1'].unique())
for dir in dirlst:
  path = os.path.join(parent_dir, dir)
  if os.path.isdir(path) == False:
    os.mkdir(path)
    print("Directory '% s' created" % dir)

In [ ]:
image_class_all = merge_image_metadata[['Image Index','Finding1']]

errors = []
root_dir= "/content/drive/MyDrive/"
for idx, csv in enumerate(csvlst,1):
  csv_data = pd.read_csv(csv)
  csv_data.drop(columns=["Unnamed: 0"])
  image_class= pd.merge(
    right=csv_data,
    left=image_class_all,
    how="inner",
    on=None,
    right_on=['Image Index'],
    left_on=['Image Index'],
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=None,
    copy=True,
    indicator=False,
    validate=None)

  for row in  range(len(image_class)):
    image = image_class.iloc[row,0]
    subdir= image_class.iloc[row,1]
    infile = os.path.join(root_dir, 'images_%02d' % (idx), 'images',image)
    outfile = os.path.join(root_dir, 'images', subdir, image)
    try:
      os.system('cp -p %s %s' % (infile, outfile))
      #shutil.copy(infile, outfile)
    except OSError as error:
      print(error)
      continue
  print("End of copying images from:", 'images_%02d' % (idx))

In [ ]:
img = cv2.imread("/content/drive/MyDrive/data/Atelectasis/00000011_006.png")
dimensions = img.shape
print("Dimensions : ",dimensions)
print("Height : ",dimensions[0])
print("Width : ",dimensions[1])
print("Channels : ",dimensions[2])

In [ ]:
image_title = "Atelectasis"
img = load_img("/content/drive/MyDrive/data/Atelectasis/00000011_006.png")

plt.imshow(img)
plt.title(image_title)
plt.show()

In [ ]:
image_title = "No Finding"
img = load_img("/content/drive/MyDrive/data/NoFinding/00010237_000.png")
plt.imshow(img)
plt.title(image_title)
plt.show()